In [27]:
import scipy.sparse
from sagemaker import image_uris
image_uris.retrieve(framework='tensorflow',region='us-east-1',version='2.12.1',image_scope='inference',instance_type='ml.c5.4xlarge')

'763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference:2.12.1-cpu'

In [28]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 520713654638.dkr.ecr.us-east-1.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [29]:
!docker pull 520713654638.dkr.ecr.us-east-1.amazonaws.com/sagemaker-tensorflow-serving:1.12.0-cpu

1.12.0-cpu: Pulling from sagemaker-tensorflow-serving

8f4ce990: Pulling fs layer 
282d9c4b: Pulling fs layer 
3db6dc03: Pulling fs layer 
c605f3f1: Pulling fs layer 
10487652: Pulling fs layer 
dde97789: Pulling fs layer 
b514193e: Pulling fs layer 
9f2d3d2a: Pulling fs layer 
e4419145: Pull complete 874kB/1.874kBBADigest: sha256:026c2a51a2e9e258481b17c110f9c9640dc010bf71463d4bd31e925ced7c6a49
Status: Downloaded newer image for 520713654638.dkr.ecr.us-east-1.amazonaws.com/sagemaker-tensorflow-serving:1.12.0-cpu
520713654638.dkr.ecr.us-east-1.amazonaws.com/sagemaker-tensorflow-serving:1.12.0-cpu


In [30]:
import scipy.sparse
from sagemaker import image_uris
image_uris.retrieve(framework='tensorflow',region='us-east-1',version='2.12.0',image_scope='training',instance_type='ml.c5.4xlarge')

[11/22/24 15:46:54] INFO     Defaulting to only available Python version: py310                   ]8;id=920402;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=295198;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py#603\603]8;;\

'763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:2.12.0-cpu-py310'

In [31]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 763104351884.dkr.ecr.us-east-1.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [32]:
!docker images

REPOSITORY                                                                  TAG          IMAGE ID       CREATED          SIZE
319840978863.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference           mondo        26210a2e8da1   57 minutes ago   4.53GB
520713654638.dkr.ecr.us-east-1.amazonaws.com/sagemaker-tensorflow-serving   1.12.0-cpu   27aee10a9f6a   5 years ago      377MB


In [ ]:
!docker build -t 319840978863.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:mondo -f Dockerfile.train .

In [34]:
!docker images

REPOSITORY                                                                  TAG          IMAGE ID       CREATED             SIZE
319840978863.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training            mondo        f75946f23fd8   4 minutes ago       7GB
319840978863.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference           mondo        26210a2e8da1   About an hour ago   4.53GB
520713654638.dkr.ecr.us-east-1.amazonaws.com/sagemaker-tensorflow-serving   1.12.0-cpu   27aee10a9f6a   5 years ago         377MB


In [38]:
%%sh
# Specify an image name
image_name=tensorflow-training
echo "image_name: ${image_name} ######################"

account=$(aws sts get-caller-identity --query Account --output text)
echo "account: ${account} ######################"

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
echo "region: ${region} ######################"

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${image_name}:2.12.0-cpu-py310-pluto-2"
echo "fullname: ${fullname} ######################"

aws ecr describe-repositories --repository-names "${image_name}"
if [ $? -ne 0 ]
then
aws ecr create-repository --repository-name "${image_name}"
fi

image_name: tensorflow-training ######################
account: 319840978863 ######################
region: us-east-1 ######################
fullname: 319840978863.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:2.12.0-cpu-py310-pluto-2 ######################



An error occurred (RepositoryNotFoundException) when calling the DescribeRepositories operation: The repository with name 'tensorflow-training' does not exist in the registry with id '319840978863'


{
    "repository": {
        "repositoryArn": "arn:aws:ecr:us-east-1:319840978863:repository/tensorflow-training",
        "registryId": "319840978863",
        "repositoryName": "tensorflow-training",
        "repositoryUri": "319840978863.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training",
        "createdAt": 1732291185.597,
        "imageTagMutability": "MUTABLE",
        "imageScanningConfiguration": {
            "scanOnPush": false
        },
        "encryptionConfiguration": {
            "encryptionType": "AES256"
        }
    }
}


In [39]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin  319840978863.dkr.ecr.us-east-1.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [40]:
!docker images

REPOSITORY                                                                  TAG          IMAGE ID       CREATED             SIZE
319840978863.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training            mondo        f75946f23fd8   9 minutes ago       7GB
319840978863.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference           mondo        26210a2e8da1   About an hour ago   4.53GB
520713654638.dkr.ecr.us-east-1.amazonaws.com/sagemaker-tensorflow-serving   1.12.0-cpu   27aee10a9f6a   5 years ago         377MB


In [41]:
!docker tag ciao:mondo 319840978863.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:mondo

Error response from daemon: No such image: ciao:mondo


In [42]:
!docker images

REPOSITORY                                                                  TAG          IMAGE ID       CREATED             SIZE
319840978863.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training            mondo        f75946f23fd8   10 minutes ago      7GB
319840978863.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference           mondo        26210a2e8da1   About an hour ago   4.53GB
520713654638.dkr.ecr.us-east-1.amazonaws.com/sagemaker-tensorflow-serving   1.12.0-cpu   27aee10a9f6a   5 years ago         377MB


In [43]:
!docker push 319840978863.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:mondo

The push refers to repository [319840978863.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training]

e5280b47: Preparing 
d88e4ba4: Preparing 
5f6e8138: Preparing 
117206ec: Preparing 
22878ecf: Preparing 
db4f6696: Preparing 
f529ae49: Preparing 
75f13e59: Preparing 
0c6c24f9: Preparing 
a1e64371: Preparing 
29631b13: Preparing 
3f03efd0: Preparing 
9b75d5ee: Preparing 
370de786: Preparing 
a16f1a7a: Preparing 
fc8740d1: Preparing 
292691fc: Preparing 
6672bf9d: Preparing 
607be3da: Preparing 
afc9b360: Preparing 
f934e4df: Preparing 
80a2a6eb: Preparing 
5f13e59: Waiting g 
c6c24f9: Waiting g 
f0a068df: Preparing 
1e64371: Waiting g 
37a36861: Preparing 
ecb4bd81: Preparing 
34b60c81: Preparing 
07d5d0ab: Preparing 
e6c8e003: Preparing 
ab822c1c: Preparing 
3609315f: Preparing 
16f1a7a: Pushed   2.248GB/2.217GBPushing    176MB/2.217GBPushing  532.5MB/871.7MBPushing  35.32MB/72.81MBmondo: digest: sha256:eec0559d8485f650829099629540e93389954e36fbdbbfaaaf2fdb26ef3e1222 size: 7459


In [ ]:
df = pd.read_csv(fr"data/input/loan_data.csv")

# Prepare the data
df_binary = df.copy()
df_binary['person_gender'] = df['person_gender'].astype(str).str.strip().str.lower().map({'female': 0, 'male': 1})
df_binary['previous_loan_defaults_on_file'] = df['previous_loan_defaults_on_file'].map({'Yes': 1, 'No': 0})

# One hot encoding
df_encoded = pd.get_dummies(df_binary, columns=['person_education'], prefix='edu', dtype=int)
df_encoded['edu_High_School'] = df_encoded['edu_High School']
df_encoded = df_encoded.drop(columns=['edu_High School'])
df_encoded = pd.get_dummies(df_encoded, columns=['person_home_ownership'], prefix="home_own", dtype=int)
df_encoded = pd.get_dummies(df_encoded, columns=['loan_intent'], dtype=int)

# Scaling
columns_to_scale = [
    'person_age', 'person_income', 'loan_amnt', 'credit_score', 'loan_int_rate'
]
scaler = StandardScaler()

df_scaled = df_encoded.copy()
df_scaled[columns_to_scale] = scaler.fit_transform(df_encoded[columns_to_scale])

# Train-test split
from sklearn.model_selection import train_test_split
y = df_scaled['loan_status']
x = df_scaled.drop(['loan_status'], axis=1)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Create a Random Forest Classifier
model = RandomForestClassifier(random_state=42)

# Train the model
model.fit(x_train, y_train)

#   predictions on the test set
y_pred = model.predict(x_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"accuracy: {accuracy}")
print(f"precision: {precision}")
print(f"recall: {recall}")
print(f"f1 score: {f1}")

In [ ]:
import pickle

# Save the model to a file
with open('loan_data_model.pkl', 'wb') as file:
    pickle.dump(model, file)

In [ ]:
import tarfile
with tarfile.open('loan_data_model.tar.gz', 'w:gz') as tar:
    tar.add('loan_data_model.pkl')

In [46]:
%%sh
# Specify an image name
image_name=tensorflow-inference
echo "image_name: ${image_name} ######################"

account=$(aws sts get-caller-identity --query Account --output text)
echo "account: ${account} ######################"

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
echo "region: ${region} ######################"

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${image_name}:2.12.0-cpu-py310-pluto-2"
echo "fullname: ${fullname} ######################"

aws ecr describe-repositories --repository-names "${image_name}"
if [ $? -ne 0 ]
then
aws ecr create-repository --repository-name "${image_name}"
fi

image_name: tensorflow-inference ######################
account: 319840978863 ######################
region: us-east-1 ######################
fullname: 319840978863.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference:2.12.0-cpu-py310-pluto-2 ######################
{
    "repositories": [
        {
            "repositoryArn": "arn:aws:ecr:us-east-1:319840978863:repository/tensorflow-inference",
            "registryId": "319840978863",
            "repositoryName": "tensorflow-inference",
            "repositoryUri": "319840978863.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference",
            "createdAt": 1732286852.334,
            "imageTagMutability": "MUTABLE",
            "imageScanningConfiguration": {
                "scanOnPush": false
            },
            "encryptionConfiguration": {
                "encryptionType": "AES256"
            }
        }
    ]
}


In [56]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 763104351884.dkr.ecr.us-east-1.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [57]:
!docker build -t 319840978863.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference:mondo -f Dockerfile.inference .

[+] Building 0.0s (0/1)                                          docker:default
[+] Building 0.2s (3/3)                                          docker:default
 => [internal] load build definition from Dockerfile.inference             0.0s
 => => transferring dockerfile: 425B                                       0.0s
 => [internal] load metadata for 763104351884.dkr.ecr.us-east-1.amazonaws  0.2s
 => [auth] sharing credentials for 763104351884.dkr.ecr.us-east-1.amazona  0.0s
[+] Building 0.3s (11/11) FINISHED                               docker:default
 => [internal] load build definition from Dockerfile.inference             0.0s
 => => transferring dockerfile: 425B                                       0.0s
 => [internal] load metadata for 763104351884.dkr.ecr.us-east-1.amazonaws  0.2s
 => [auth] sharing credentials for 763104351884.dkr.ecr.us-east-1.amazona  0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B         

In [58]:
!docker images

REPOSITORY                                                                  TAG          IMAGE ID       CREATED          SIZE
319840978863.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference           mondo        763ef1a70752   12 minutes ago   4.53GB
319840978863.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training            mondo        f75946f23fd8   38 minutes ago   7GB
319840978863.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference           <none>       26210a2e8da1   2 hours ago      4.53GB
520713654638.dkr.ecr.us-east-1.amazonaws.com/sagemaker-tensorflow-serving   1.12.0-cpu   27aee10a9f6a   5 years ago      377MB


In [59]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 319840978863.dkr.ecr.us-east-1.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [60]:
!docker push 319840978863.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference:mondo

The push refers to repository [319840978863.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference]

b4272b3a: Preparing 
7d0cb892: Preparing 
c386816e: Preparing 
f1a769d3: Preparing 
37bf1d61: Preparing 
8df97c44: Preparing 
764747b4: Preparing 
1a68f579: Preparing 
2eef5eea: Preparing 
b5bcc575: Preparing 
5e5ce62f: Preparing 
757a305d: Preparing 
c7cf6f28: Preparing 
30092134: Preparing 
fb153852: Preparing 
0903db8c: Preparing 
19dec72a: Preparing 
df04f233: Preparing 
f2dbc490: Preparing 
ba0431f9: Preparing 
71536788: Preparing 
994107ae: Preparing 
3a4f83e7: Preparing 
d6748243: Preparing 
f2c1e372: Preparing 
cd2b5d6d: Preparing 
ca73c74f: Layer already exists mondo: digest: sha256:2bfe6b7f4fa8358e37a9c0b84cd603b51ccb89d1e9ac61f21508a2cc06278cfd size: 5972


In [63]:

from sagemaker import get_execution_role
import boto3
import datetime
from time import gmtime, strftime

# https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-deployment.html

my_session = boto3.session.Session()
aws_region = my_session.region_name

sagemaker_client = boto3.client('sagemaker', region_name=aws_region)

sagemaker_role = get_execution_role()

model_name = '2modelLarocca'

# Create model
create_model_response = sagemaker_client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = sagemaker_role,
    PrimaryContainer = {
        'Image': '319840978863.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference:mondo',
        'ModelDataUrl': 's3://itsar123-larocca/data/output/loan_data_model.tar.gz',
    })




In [67]:
# Create an endpoint config name. Here we create one based on the date  
# so it we can search endpoints based on creation time.
endpoint_config_name = 'prima-api-inferenza-larocca2'

instance_type = 'ml.m5.large'

endpoint_config_response = sagemaker_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    # You will specify this name in a CreateEndpoint request.
    # List of ProductionVariant objects, one for each model that you want to host at this endpoint.
    ProductionVariants=[
        {
            "VariantName": "variant1", # The name of the production variant.
            "ModelName": model_name,
            "InstanceType": instance_type, # Specify the compute instance type.
            "InitialInstanceCount": 1 # Number of instances to launch initially.
        }
    ]
)

print(f"Created EndpointConfig: {endpoint_config_response['EndpointConfigArn']}")

# The name of the endpoint. The name must be unique within an AWS Region in your AWS account.
endpoint_name = 'prima-api-inferenza-larocca2'

create_endpoint_response = sagemaker_client.create_endpoint(
                                            EndpointName=endpoint_name, 
                                            EndpointConfigName=endpoint_config_name)


Created EndpointConfig: arn:aws:sagemaker:us-east-1:319840978863:endpoint-config/prima-api-inferenza-larocca2


In [68]:
!chmod -x pippo.sh